In [2]:
!pip install implicit

  Stored in directory: C:\Users\marchenko-i-u\AppData\Local\pip\Cache\wheels\1b\48\b1\1aebe3acc3afb5589e72d3e7c3ffc3f637dc4721c1a974dff7
Successfully built implicit


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [46]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [19]:
data_train.shape,data_test.shape

((2278490, 12), (118314, 12))

In [278]:
result = pd.read_csv('predictions_basic.csv')

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [103]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(items_weights['item_id'].values)
    recs = np.random.choice(items, size=n, replace=False, p=np.array(items_weights['weight'].values))
    
    return recs.tolist() 

In [63]:
%%time

# your_code
items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()

sum_ = np.log1p(items_weights['sales_value']).sum()
items_weights['weight'] = np.log1p(items['sales_value'])/sum_
items_weights.head()

Wall time: 136 ms


,item_id,sales_value,weight
0,25671,20.94,0.000013
1,26081,0.99,0.000003
2,26093,1.59,0.000004
3,26190,1.54,0.000004
4,26355,1.98,0.000005


In [64]:
items_weights['weight'].sum()

1.0

In [335]:
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,1,random_recommendation_precision,popular_recommendation_precision,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[8157117, 870291, 1846901, 891748, 1038560]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]",0.0,0.0,0.2,"[15481763, 828878, 12782154, 1126207, 5592653]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1000965, 10121560, 9878622, 13512901, 13039531]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]",0.0,0.0,0.0,"[9857322, 1244154, 1110229, 842405, 973340]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [305]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[8157117, 870291, 1846901, 891748, 1038560]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[1000965, 10121560, 9878622, 13512901, 13039531]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [306]:
def col_to_str(col, spl):
    if(spl!=''):
        result[col] = result[col].apply(lambda x: x.replace('[', '').replace(']', '').split(spl))
    else:
        result[col] = result[col].apply(lambda x: x.replace('[', '').replace(']', '').split())

In [307]:
col_to_str('actual','')
col_to_str('random_recommendation',', ')
col_to_str('popular_recommendation',', ')
col_to_str('itemitem',', ')
col_to_str('cosine',', ')
col_to_str('tfidf',', ')
col_to_str('own_purchases',', ')

In [308]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[8157117, 870291, 1846901, 891748, 1038560]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1000965, 10121560, 9878622, 13512901, 13039531]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [322]:
# your_code
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
   
    return precision

In [333]:
result['random_recommendation_precision'] = result['user_id'].\
                    apply(lambda x: 
                          precision_at_k(result.loc[result['user_id']==x,'random_recommendation'].values[0], result.loc[result['user_id']==x,'actual'].values[0], k=5)
                    )
result['random_recommendation_precision'].value_counts()

0.0    2036
0.2       6
Name: random_recommendation_precision, dtype: int64

In [334]:
result['popular_recommendation_precision'] = result['user_id'].\
                    apply(lambda x: 
                          precision_at_k(result.loc[result['user_id']==x,'popular_recommendation'].values[0], result.loc[result['user_id']==x,'actual'].values[0], k=5)
                    )

result['popular_recommendation_precision'].value_counts()

0.0    957
0.2    681
0.4    313
0.6     86
0.8      5
Name: popular_recommendation_precision, dtype: int64

In [336]:
result['weighted_recommendation_precision'] = result['user_id'].\
                    apply(lambda x: 
                          precision_at_k(result.loc[result['user_id']==x,'weighted_random_recommendation'].values[0], result.loc[result['user_id']==x,'actual'].values[0], k=5)
                    )
result['weighted_recommendation_precision'].value_counts()

C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


0.0    2042
Name: weighted_recommendation_precision, dtype: int64

In [337]:
result['itemitem_precision'] = result['user_id'].\
                    apply(lambda x: 
                          precision_at_k(result.loc[result['user_id']==x,'itemitem'].values[0], result.loc[result['user_id']==x,'actual'].values[0], k=5)
                    )
result['itemitem_precision'].value_counts()

0.0    1001
0.2     641
0.4     310
0.6      77
0.8      12
1.0       1
Name: itemitem_precision, dtype: int64

In [338]:
result['cosine_precision'] = result['user_id'].\
                    apply(lambda x: 
                          precision_at_k(result.loc[result['user_id']==x,'cosine'].values[0], result.loc[result['user_id']==x,'actual'].values[0], k=5)
                    )
result['cosine_precision'].value_counts()

0.0    995
0.2    635
0.4    302
0.6     94
0.8     14
1.0      2
Name: cosine_precision, dtype: int64

In [339]:
result['tfidf_precision'] = result['user_id'].\
                    apply(lambda x: 
                          precision_at_k(result.loc[result['user_id']==x,'tfidf'].values[0], result.loc[result['user_id']==x,'actual'].values[0], k=5)
                    )
result['tfidf_precision'].value_counts()

0.0    975
0.2    641
0.4    299
0.6    103
0.8     22
1.0      2
Name: tfidf_precision, dtype: int64

In [340]:
result['own_purchases_precision'] = result['user_id'].\
                    apply(lambda x: 
                          precision_at_k(result.loc[result['user_id']==x,'own_purchases'].values[0], result.loc[result['user_id']==x,'actual'].values[0], k=5)
                    )
result['own_purchases_precision'].value_counts()

0.0    817
0.2    643
0.4    371
0.6    176
0.8     31
1.0      3
0.5      1
Name: own_purchases_precision, dtype: int64

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ- {2000, 5000, 10000} товаров. Постройте график зависимости precision@5 от топ-*
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$. Постройте график зависимости precision@5 от $K$ 

In [ ]:
# your_code